Install required module by the package management tool pip

In [1]:
pip install moviepy

Note: you may need to restart the kernel to use updated packages.


5. import dependencies for exercise 1.2

In [2]:
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
import pandas as pd

6. Improvements for Exercise 1.1 and 1.2

In the previous part, the first captured frame (initial frame) is regarded as the baseline image (i.e., background frame). In the method cv2.absdiff(), the difference between the baseline and the gray frame is calculated. Thus, I recommend using median frame as the background frame instead of initial frame in this part.

In [3]:
# define a function named median_frame with one parameter named path
def median_frame(path):
    
    # creage a VideoCapture object and read the frames from an input file with a given path
    video = cv2.VideoCapture(path)
    
    # generate 50 indexes based on 50 random frames for median
    indexes = video.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    
    # initialise an empty array to store frames
    frames = []
    
    # iterate each index and read video frame
    for i in indexes:
        video.set(cv2.CAP_PROP_POS_FRAMES, i)
        check, frame = video.read()
        # append new frame to an array named frames
        frames.append(frame)
        
    # calculate the median of 50 random frames
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    
    # the function returns the median frame
    return median_frame

The function backgroundDetection has the same function to the part 3 code. In other words, the function backgroundDetection can also detect and track moving cars in the video recording. In addition, the function backgroundDetection has its additional function, which is to return the value for number of cars go into city center. The value of num_car_into_city_center will be used for exercise 1.2 to calculate the value of cars per minute.

In [4]:
# define a function named backgroundDetection with three parameters
def backgroundDetection(path, numframes, rect_loc):
    
    # creage a VideoCapture object and read the frames from an input file with a given path
    video = cv2.VideoCapture(path)

    # get the height of the frame which detect cars at the main street only
    height = int(video.get(4))

    # get the median frame as the background frame based on the aforementioned function named median_frame
    background = median_frame(path)
    
    # convert the background frame in gray scale
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    
    # initialise the count as the value of 0
    count = 0
    
    # initialise the numframes as the same value to parameter numframes
    numframes = numframes

    # initiaise the number of cars go into city center as the value of 0
    num_car_into_city_center = 0
    
    # create two empty arrays to store the x and y locations of the previous and current rectangles
    current_rect = []
    previous_rect = []
    
    # read through the video frame until video is completed or the user press 'q'
    while (video.isOpened()):
        check, frame = video.read()
        if check == True:
            count += 1
            
            # make a copy of the original frame
            original_frame = frame.copy()
            
            # convert frame to grayscale
            gray_scale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # checks whether frame count has reached a multiple of numframes
            if count % numframes == 0 or count == 1:
                difference_array = []
                
            # calculates absolute difference between gray scale and background
            abs_diff = cv2.absdiff(gray_scale, background)
            
            # converts the difference to binary using thresholding
            check, thres = cv2.threshold(abs_diff, 50, 255, cv2.THRESH_BINARY)
            
            # enlarge frame to increase contour detection
            enlarge_frame = cv2.dilate(thres, None, iterations=2)
            
            # append the final result into the array named difference_array
            difference_array.append(enlarge_frame)
            
            # check if difference_array has reached the required length
            
            if len(difference_array) == numframes:
                # calculate the sum the frames in the difference_array
                sum_num_frames = sum(difference_array)
                
                # save contours around the white enlarged areas
                contours, hierarchy = cv2.findContours(sum_num_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # iterate every contour
                for i in range(len(contours)):
                    
                    # filter out any small contours that does not belong to a car shape
                    if cv2.contourArea(contours[i]) < 500:
                        continue
                    (x, y, w, h) = cv2.boundingRect(contours[i])


                    # draw green rectangles to highlight car shapes
                    if y > height / 2 and w * h > 6000:
                        cv2.rectangle(original_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                        
                        # append the locations of the rectangles to an array named current_rect
                        current_rect.append((x, y, w, h))

                # find the minimum length of the previous and current array of rectangles
                min_len = min(len(current_rect), len(previous_rect))
                for i in range(min_len):
                    if current_rect[i][0] > 400 and current_rect[i][0] < rect_loc and current_rect[i][1] > 350 and current_rect[i][1] < 400 and current_rect[i][0] < previous_rect[i][0]:
                        
                        # calculate the number of cars go into the city center
                        num_car_into_city_center += 1
                        
                        # print current number of cars go into the city center
                        print(num_car_into_city_center)

                # set the previous array of rectangles equal to the current array of rectangles
                previous_rect = current_rect
                
                # reset current array of rectangles
                current_rect = []

                # display the image
                cv2.imshow('Detected Objects', original_frame)
                
                # detect key event every 100ms , if the user press 'q', then stop the program 
                if cv2.waitKey(100) & 0xFF == ord('q'):
                    break
        else:
            break
            
            
    # after the loop above, release the video object
    video.release()
    
    # and destroy all windows
    cv2.destroyAllWindows() 
    
    # return the value for number of cars go into city center 
    return num_car_into_city_center

7. calculate **Total number of cars** and **Cars per minute** 

In [5]:
# get the value for numbers of cars go into city center on video 1 
num_car_into_city_center_1 = backgroundDetection('Traffic_Laramie_1.mp4', 4, 435)

# get the value for numbers of cars go into city center on video 2
num_car_into_city_center_2 = backgroundDetection('Traffic_Laramie_2.mp4', 5, 440)

1
2
3
4
5
6
1
2
3
4


In [6]:
# load the first video by VideoFileClip method
video1 = VideoFileClip('Traffic_Laramie_1.mp4')

# calculate the first video duration in seconds and print the result
video1_duration_in_seconds = video1.duration

print(video1_duration_in_seconds)

177.92000000000002


In [7]:
# calculate the first video duration in minutes and print the result
video1_duration_in_minutes = video1_duration_in_seconds / 60

print(video1_duration_in_minutes)

2.9653333333333336


In [8]:
# calculate the value of cars per minute on video 1
video1_cars_per_minute = num_car_into_city_center_1 / video1_duration_in_minutes

print(video1_cars_per_minute)

2.0233812949640284


In [9]:
# load the second video by VideoFileClip method
video2 = VideoFileClip('Traffic_Laramie_2.mp4')

# calculate the second video duration in seconds and print the result
video2_duration_in_seconds = video2.duration

print(video2_duration_in_seconds)

105.68


In [10]:
# calculate the second video duration in minutes and print the result
video2_duration_in_minutes = video2_duration_in_seconds / 60

print(video2_duration_in_minutes)

1.7613333333333334


In [11]:
# calculate the value of cars per minute on video 2
video2_cars_per_minute = num_car_into_city_center_2 / video2_duration_in_minutes

print(video2_cars_per_minute)

2.271006813020439


In [12]:
# cumulate the required table data and store it as a variable named required_table_data
required_table_data = [["Traffic_Laramie_1.mp4", num_car_into_city_center_1, video1_cars_per_minute], 
                       ["Traffic_Laramie_2.mp4", num_car_into_city_center_2, video2_cars_per_minute]]

# print the required table as pandas dataframe
pd.DataFrame(required_table_data, columns=["file", "Total number of cars", "Cars per minute"])

,file,Total number of cars,Cars per minute
0,Traffic_Laramie_1.mp4,6,2.023381
1,Traffic_Laramie_2.mp4,4,2.271007


Thus, we can fill the following table with the data.
                        
                        Table 1

| **file** | **Total number of cars** | **Cars per minute** |
|:----------:|--------------------:|:--------:|
| **Traffic_Laramie_1.mp4**     | `6 `            |   2.023381    |
| **Traffic_Laramie_2.mp4**     | ` 4`            |  2.271007     |

For more details about the application, please read the ISP final report of exercise 1.1 and 1.2.